## Experiments for binary treatment effect estimation comparison

In [1]:
import sys, os

# add the project root to sys.path
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root not in sys.path:
    sys.path.insert(0, root)

from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')

import CausalEGM as cegm
# import the module
from models import *

import numpy as np
import pickle
import os
from tqdm import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)



np.random.seed(42)
n_tr = 1000
n_p = 1000

nI = 2
nX = 2
nO = 2
nS= 2
p = nI+nX+nO+nS
ate = 2
beta_cov = 0
strength_instr = 1
strength_conf = 1
strength_outcome = 1
binary_intervention=True

## Example of hyperparameter tuning

In [2]:
from functools import lru_cache

def tune_and_eval(model_name,
                  X_train, t_train, y_train,
                  X_val,   t_val,   y_val,
                  X_test,  t_test,  y_test,
                  provided_params=None,
                  n_trials=10):
    """
    If best_params is None: runs Optuna, returns (ITE_array, best_params).
    If best_params is given: skips Optuna, returns ITE_array only.
    """
    # 1) hyperparam search
    if provided_params is None:
        import optuna
        study = optuna.create_study(direction="minimize",
                                    study_name=f"{model_name}_tune")
        def objective(trial):
            # common
            lr     = trial.suggest_loguniform("lr", 1e-5, 1e-2)
            wd     = trial.suggest_loguniform("wd", 1e-5, 1e-2)
            bs     = trial.suggest_categorical("bs", [32, 128, 256])
            epochs = trial.suggest_int("epochs", 300, 800)

            # model‐specific
            if model_name == "tarnet":
                rep1 = trial.suggest_int("rep1", 20, 50 )
                rep2 = trial.suggest_int("rep2", 50, 100)
                head = trial.suggest_int("head", 50, 100)
                drop = trial.suggest_uniform("drop", 0.0, 0.001)
                trainer = TARNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop)

            elif model_name == "cfrnet":
                rep1   = trial.suggest_int("rep1", 50, 200)
                rep2   = trial.suggest_int("rep2", 50, 200)
                head   = trial.suggest_int("head", 50, 200)
                drop   = trial.suggest_uniform("drop", 0.0, 0.001)
                ipm_w  = trial.suggest_loguniform("ipm_weight", 0.01, 10.0)
                trainer = CFRNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop, ipm_w)

            elif model_name == "cevae":
                ld = trial.suggest_int("latent_dim", 10, 200)
                hd = trial.suggest_int("hidden_dim", 20, 400)
                nl = trial.suggest_int("num_layers", 2, 5)     # note: 2→5 to avoid pop error
                ns = trial.suggest_categorical("num_samples", [10,50,100,200])
                trainer = CEVAETrainer(X_train.shape[1], ld, hd, nl, ns)

            else:  # dragonnet
                sh = trial.suggest_int("shared_hidden", 50, 200)
                oh = trial.suggest_int("outcome_hidden", 50, 200)
                trainer = DragonNetTrainer(X_train.shape[1], sh, oh)

            return trainer.fit(
                X_train, t_train, y_train,
                X_val,   t_val,   y_val,
                lr=lr, weight_decay=wd,
                batch_size=bs, epochs=epochs
            )

        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print(f"🔍 Best params for {model_name}: {best_params}")
    else:
        best_params = provided_params
    # 2) retrain on combined train+val
    X_trn = np.vstack([X_train, X_val])
    t_trn = np.concatenate([t_train, t_val])
    y_trn = np.concatenate([y_train, y_val])

    if model_name == "tarnet":
        trainer = TARNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop']
        )
    elif model_name == "cfrnet":
        trainer = CFRNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop'],
            best_params['ipm_weight']
        )
    elif model_name == "cevae":
        trainer = CEVAETrainer(
            X_trn.shape[1],
            best_params['latent_dim'],
            best_params['hidden_dim'],
            best_params['num_layers'],
            best_params['num_samples']
        )
    else:
        trainer = DragonNetTrainer(
            X_trn.shape[1],
            best_params['shared_hidden'],
            best_params['outcome_hidden']
        )

    trainer.fit(
        X_trn, t_trn, y_trn,
        X_test, t_test, y_test,
        lr=best_params['lr'],
        weight_decay=best_params['wd'],
        batch_size=best_params['bs'],
        epochs=best_params['epochs']
    )

    if model_name == "cevae":
        ite = trainer.predict(X_test)
    else:
        y0p, y1p = trainer.predict(X_test)
        ite = y1p - y0p

    return (ite, best_params) if provided_params is None else ite


## Fitting synthetic data generated by causl

### Data generation

In [3]:
nrep = 30  # Number of repetitions
n_tr = 1000  # Training sample size
n_val = 400
n_te = 400  # Testing sample size
strength_instr_values = np.arange(1,4.5,1)  # Varying strength of instrumental variables
nI = 4 # Fixed number of instrumental variables
nX = 3
nO = 3
nS = 0
binary_intervention = True
num_iters = 800  # Fixed number of training iterations
ate = 2
strength_conf = 1
strength_outcome = 1

# Initialize tracker for strength_instr
tracker = {strength_instr: {"fr": [], "dr": [], "causalegm":[], "tarnet":[], "cfrnet":[], "cevae":[], "dragonnet":[]}
           for strength_instr in strength_instr_values}
best_hps = {model: None for model in ["tarnet","cfrnet","cevae","dragonnet"]}
# Begin loop over strength_instr
for strength_instr in strength_instr_values:
    print(f"Running experiments for strength_instr = {strength_instr}")
    p = nI + nX + nO + nS  # Update the number of covariates
    
    for rep in tqdm(range(nrep)):
        # Generate training and testing data
        df_tr = generate_data_causl(n=n_tr, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)
        z_tr = torch.tensor(df_tr[[f"X{i}" for i in range(1, p + 1)]].values, dtype=torch.float32)
        x_tr = torch.tensor(df_tr['A'].values, dtype=torch.int32).view(-1, 1) if binary_intervention else \
            torch.tensor(df_tr['A'].values, dtype=torch.float32).view(-1, 1)
        y_tr = torch.tensor(df_tr['y'].values, dtype=torch.float32).view(-1, 1)
        
        z_tr_np = df_tr[[f"X{i}" for i in range(1, p + 1)]].values
        x_tr_np = df_tr['A'].values
        y_tr_np = df_tr['y'].values

        df_val = generate_data_causl(n=n_val, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)


        z_val_np = df_val[[f"X{i}" for i in range(1, p + 1)]].values
        x_val_np = df_val['A'].values
        y_val_np = df_val['y'].values

        df_te = generate_data_causl(n=n_te, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)

        z_te_np = df_te[[f"X{i}" for i in range(1, p + 1)]].values
        x_te_np = df_te['A'].values
        y_te_np = df_te['y'].values
        z_te = torch.tensor(z_te_np, dtype=torch.float32)

        model = Frengression(x_dim = x_tr.shape[1], y_dim = 1, z_dim =z_tr.shape[1], 
                             noise_dim=1, num_layer=3, hidden_dim=100, 
                             device=device, x_binary=binary_intervention, z_binary_dims=0)

        # Train Frengression model
        model.train_y(x=x_tr,
                      z=z_tr, 
                      y=y_tr, 
                      num_iters=num_iters, lr=1e-4, print_every_iter=1000)

        # Sample model distributions
        P0 = model.sample_causal_margin(torch.tensor([0], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        P1 = model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        ate_fr = np.mean(P1) - np.mean(P0)

        # DR Estimation
        ate_dr, _ = dr_ate(x_tr_np, y_tr_np, z_tr_np ,x_te_np, y_te_np, z_te_np)

        for model in ["tarnet","cfrnet","cevae","dragonnet"]:
            if rep == 0:
                ite, best_hps[model] = tune_and_eval(
                    model,
                    z_tr_np, x_tr_np, y_tr_np,
                    z_val_np, x_val_np, y_val_np,
                    z_te_np, x_te_np,y_te_np,
                    provided_params=None,
                    n_trials=10
                )
            else:
                ite = tune_and_eval(
                    model,
                    z_tr_np, x_tr_np, y_tr_np,
                    z_val_np, x_val_np, y_val_np,
                    z_te_np, x_te_np,y_te_np,
                    provided_params=best_hps[model]
                )
            tracker[strength_instr][model].append(ite.mean())


        cegm_params = {'dataset': 'Semi_acic', 
                        'output_dir': '.', 
                        'v_dim': z_tr.shape[1], 
                        'z_dims': [1, 1, 1, 1], 
                        'lr': 0.0002, 
                        'alpha': 1, 
                        'beta': 1, 
                        'gamma': 10, 
                        'g_d_freq': 5, 
                        'g_units': [64, 64, 64, 64, 64], 
                        'e_units': [64, 64, 64, 64, 64], 
                        'f_units': [64, 32, 8], 
                        'h_units': [64, 32, 8], 
                        'dz_units': [64, 32, 8], 
                        'dv_units': [64, 32, 8], 'save_res': False, 'save_model': False, 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True}
        egm_model = cegm.CausalEGM(params=cegm_params, random_seed=42)
        egm_model.train(data=[x_tr,y_tr,z_tr],n_iter=800, verbose=False)
        ate_causalegm=egm_model.getCATE(z_te).mean()

        # Log results
        tracker[strength_instr]["fr"].append(ate_fr)
        tracker[strength_instr]["dr"].append(ate_dr)
        tracker[strength_instr]["causalegm"].append(ate_causalegm)
    
        

Running experiments for strength_instr = 1.0


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1: loss 2.4448,	loss_y 1.6538, 1.6757, 0.0438,	loss_eta 0.7911, 0.8234, 0.0647


[I 2025-04-26 07:40:03,142] A new study created in memory with name: tarnet_tune
[I 2025-04-26 07:40:08,053] Trial 0 finished with value: 0.8636188507080078 and parameters: {'lr': 0.0011671786219347366, 'wd': 3.367012757965983e-05, 'bs': 256, 'epochs': 732, 'rep1': 21, 'rep2': 74, 'head': 72, 'drop': 0.0009248526122718238}. Best is trial 0 with value: 0.8636188507080078.
[I 2025-04-26 07:40:18,246] Trial 1 finished with value: 0.9005096554756165 and parameters: {'lr': 0.008688445937566635, 'wd': 9.42695744836571e-05, 'bs': 32, 'epochs': 535, 'rep1': 23, 'rep2': 99, 'head': 83, 'drop': 0.0009787880343823615}. Best is trial 0 with value: 0.8636188507080078.
[I 2025-04-26 07:40:24,406] Trial 2 finished with value: 0.8599570393562317 and parameters: {'lr': 0.0007625449083810562, 'wd': 0.001551200794390357, 'bs': 128, 'epochs': 780, 'rep1': 22, 'rep2': 78, 'head': 54, 'drop': 9.603615402211196e-05}. Best is trial 2 with value: 0.8599570393562317.
[I 2025-04-26 07:40:37,465] Trial 3 finished

🔍 Best params for tarnet: {'lr': 5.6456643205386084e-05, 'wd': 0.0007383829600879233, 'bs': 128, 'epochs': 594, 'rep1': 43, 'rep2': 71, 'head': 60, 'drop': 0.0009439182532494132}


...

The average treatment effect (ATE) is 1.5856658
Epoch 1: loss 2.4312,	loss_y 1.6956, 1.7220, 0.0528,	loss_eta 0.7356, 0.7811, 0.0909


INFO 	 Training with 11 minibatches per epoch
INFO 	 Evaluating 1 minibatches
2025-04-26 12:22:47.075900: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 12:22:47.076210: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant

The average treatment effect (ATE) is 1.6054076
Epoch 1: loss 2.3800,	loss_y 1.6357, 1.6658, 0.0602,	loss_eta 0.7443, 0.7776, 0.0666


INFO 	 Training with 11 minibatches per epoch
INFO 	 Evaluating 1 minibatches
2025-04-26 12:25:19.331807: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 12:25:19.332084: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant

The average treatment effect (ATE) is 1.6432047


In [4]:
import json
output_dir = "result/binary"
os.makedirs(output_dir, exist_ok=True)
tracker_serializable = {
    str(k): [float(x) for x in v_dict.get("fr",[])] 
              + []  # (we'll overwrite below) 
    for k, v_dict in tracker.items()
}
# actually build full dict:
tracker_serializable = {
    str(k): {
        model: [float(x) for x in v_list]
        for model, v_list in v_dict.items()
    }
    for k, v_dict in tracker.items()
}

# 3) write it out
with open(os.path.join(output_dir, "synthetic_1k.json"), "w") as f:
    json.dump(tracker_serializable, f, indent=4)


## IHDP

In [ ]:
# Experiment parameters
binary_intervention = True
num_iters = 400
p = 25
z_binary_dims = 19
path = '/Users/linyingyang/Documents/Project/frengression/frengression/data_causl'

# Initialize tracker for valid trials

valid_trials = 0
max_trials = 50  # We want results from 50 valid trials
trial = 0


tracker = {"fr": [], "dr": [], "causalegm":[], "tarnet":[], "cfrnet":[], "cevae":[], "dragonnet":[]}
best_hps = {model: None for model in ["tarnet","cfrnet","cevae","dragonnet"]}

while valid_trials < max_trials:
    print(f"Checking trial = {trial}")
    df_tr, df_te = process_data(path=path, trial=trial)

    # Skip this trial if any y_factual in df_tr exceeds 20
    if (df_tr['y_factual'] > 20).any():
        print(f"Skipping trial {trial} because y_factual > 20")
        trial += 1
        continue

    print(f"Running on valid trial = {trial}")
    
    # Prepare tensors for training
    z_tr = torch.tensor(df_tr[[f"X{i}" for i in range(1, p + 1)]].values, dtype=torch.float32)
    x_tr = torch.tensor(df_tr['treatment'].values, dtype=torch.float32).view(-1, 1)
    y_tr = torch.tensor(df_tr['y_factual'].values, dtype=torch.float32).view(-1, 1)
    ate_sample = torch.tensor(np.mean(df_tr['mu1'].values - df_tr['mu0'].values), dtype=torch.float32).view(-1, 1)
    
    # DR ATE estimation
    z_tr_np = df_tr[[f"X{i}" for i in range(1, p + 1)]].values
    x_tr_np = df_tr['treatment'].values
    y_tr_np = df_tr['y_factual'].values

    z_te_np = df_te[[f"X{i}" for i in range(1, p + 1)]].values
    z_te = torch.tensor(df_te[[f"X{i}" for i in range(1, p + 1)]].values, dtype=torch.float32)
    x_te_np = df_te['treatment'].values
    y_te_np = df_te['y_factual'].values
    hat_dr, _ = dr_ate(x_tr_np, y_tr_np, z_tr_np, x_te_np, y_te_np, z_te_np)

    # Initialize Frengression model
    model = Frengression(x_tr.shape[1], y_tr.shape[1], z_tr.shape[1], 
                         noise_dim=1, num_layer=3, hidden_dim=500, 
                         device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                         x_binary=binary_intervention, z_binary_dims=0)
    

    model.train_y(x_tr, z_tr, y_tr, num_iters=400, lr=1e-4, print_every_iter=100)


    # Sample model distributions
    P0 = model.predict_causal(torch.tensor([0], dtype=torch.int32), sample_size=1000).numpy()
    P1 = model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=1000).numpy().reshape(-1, 1)
    ate_fr = np.mean(P1) - np.mean(P0)


    # DR Estimation
    ate_dr, _ = dr_ate(x_tr_np, y_tr_np, z_tr_np ,x_te_np, y_te_np, z_te_np)

    for model in ["tarnet","cfrnet","cevae","dragonnet"]:
        if valid_trials == 0:
            ite, best_hps[model] = tune_and_eval(
                model,
                z_tr_np, x_tr_np, y_tr_np,
                z_tr_np, x_tr_np, y_tr_np,
                z_te_np, x_te_np,y_te_np,
                provided_params=None,
                n_trials=20
            )
        else:
            ite = tune_and_eval(
                model,
                z_tr_np, x_tr_np, y_tr_np,
                z_tr_np, x_tr_np, y_tr_np,
                z_te_np, x_te_np,y_te_np,
                provided_params=best_hps[model]
            )
        tracker[model].append(ite.mean())


    cegm_params = {'dataset': 'Semi_acic', 
                    'output_dir': '.', 
                    'v_dim': z_tr.shape[1], 
                    'z_dims': [1, 1, 1, 1], 
                    'lr': 0.0002, 
                    'alpha': 1, 
                    'beta': 1, 
                    'gamma': 10, 
                    'g_d_freq': 5, 
                    'g_units': [64, 64, 64, 64, 64], 
                    'e_units': [64, 64, 64, 64, 64], 
                    'f_units': [64, 32, 8], 
                    'h_units': [64, 32, 8], 
                    'dz_units': [64, 32, 8], 
                    'dv_units': [64, 32, 8], 'save_res': False, 'save_model': False, 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True}
    egm_model = cegm.CausalEGM(params=cegm_params, random_seed=42)
    egm_model.train(data=[x_tr,y_tr,z_tr],n_iter=800, verbose=False)
    ate_causalegm=egm_model.getCATE(z_te).mean()
    tracker['causalegm'].append(ate_causalegm)
    tracker['fr'].append(ate_fr)
    tracker['dr'].append(ate_dr)

    # Increment valid trials counter and move to the next trial
    valid_trials += 1
    trial += 1



Checking trial = 0
Running on valid trial = 0
Epoch 1: loss 3.8401,	loss_y 3.1015, 3.1212, 0.0394,	loss_eta 0.7386, 0.7636, 0.0499
Epoch 400: loss 1.0217,	loss_y 0.4154, 0.9185, 1.0062,	loss_eta 0.6064, 1.0806, 0.9484


[I 2025-04-26 17:04:09,260] A new study created in memory with name: tarnet_tune
[I 2025-04-26 17:04:14,035] Trial 0 finished with value: 1.0104341506958008 and parameters: {'lr': 0.0001452698617364503, 'wd': 0.0012526263115156919, 'bs': 128, 'epochs': 659, 'rep1': 46, 'rep2': 78, 'head': 88, 'drop': 0.0009131326037892298}. Best is trial 0 with value: 1.0104341506958008.
[I 2025-04-26 17:04:17,774] Trial 1 finished with value: 0.2894759178161621 and parameters: {'lr': 0.000878751123383489, 'wd': 0.003866714596225409, 'bs': 256, 'epochs': 692, 'rep1': 24, 'rep2': 95, 'head': 66, 'drop': 0.0003449875654759119}. Best is trial 1 with value: 0.2894759178161621.
[I 2025-04-26 17:04:22,465] Trial 2 finished with value: 3.1734859943389893 and parameters: {'lr': 4.041837938030283e-05, 'wd': 0.007880803908061782, 'bs': 32, 'epochs': 344, 'rep1': 21, 'rep2': 81, 'head': 65, 'drop': 3.0919445356371586e-05}. Best is trial 1 with value: 0.2894759178161621.
[I 2025-04-26 17:04:27,419] Trial 3 finishe

🔍 Best params for tarnet: {'lr': 0.0014377451537955332, 'wd': 0.00014915824534616717, 'bs': 32, 'epochs': 747, 'rep1': 46, 'rep2': 71, 'head': 79, 'drop': 0.0004129491998637863}


[I 2025-04-26 17:06:27,707] A new study created in memory with name: cfrnet_tune
[I 2025-04-26 17:06:36,546] Trial 0 finished with value: 2.7676749229431152 and parameters: {'lr': 2.2039127359395403e-05, 'wd': 1.217439232042258e-05, 'bs': 32, 'epochs': 413, 'rep1': 149, 'rep2': 66, 'head': 136, 'drop': 0.0009169815855689647, 'ipm_weight': 1.6039062901497152}. Best is trial 0 with value: 2.7676749229431152.
[I 2025-04-26 17:06:43,674] Trial 1 finished with value: 2.4398536682128906 and parameters: {'lr': 4.605096578840947e-05, 'wd': 0.0022885998770808902, 'bs': 128, 'epochs': 502, 'rep1': 183, 'rep2': 118, 'head': 159, 'drop': 0.0007848379361446074, 'ipm_weight': 9.034406373662197}. Best is trial 1 with value: 2.4398536682128906.
[I 2025-04-26 17:06:48,988] Trial 2 finished with value: 3.7480759620666504 and parameters: {'lr': 1.4678047962851243e-05, 'wd': 0.0011784882960546742, 'bs': 256, 'epochs': 531, 'rep1': 157, 'rep2': 73, 'head': 89, 'drop': 1.616635189756932e-05, 'ipm_weight': 0

🔍 Best params for cfrnet: {'lr': 0.0010446371729586333, 'wd': 6.675134668318002e-05, 'bs': 128, 'epochs': 781, 'rep1': 86, 'rep2': 132, 'head': 123, 'drop': 4.620506851541958e-06, 'ipm_weight': 0.5146342262389219}


...

2025-04-26 18:57:58.910861: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 18:57:58.911143: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Skipping trial 116 because y_factual > 20
Checking trial = 117
Skipping trial 117 because y_factual > 20
Checking trial = 118
Skipping trial 118 because y_factual > 20
Checking trial = 119
Skipping trial 119 because y_factual > 20
Checking trial = 120
Running on valid trial = 120
Epoch 1: loss 6.2656,	loss_y 5.4811, 5.4992, 0.0362,	loss_eta 0.7845, 0.8033, 0.0377
Epoch 400: loss 1.0095,	loss_y 0.4146, 0.9762, 1.1233,	loss_eta 0.5949, 1.0521, 0.9143


INFO 	 Training with 6 minibatches per epoch
INFO 	 Evaluating 1 minibatches


The average treatment effect (ATE) is 0.8935405


2025-04-26 19:00:10.294492: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-26 19:00:10.294801: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

In [247]:
import json, os

output_dir = "result/binary"
os.makedirs(output_dir, exist_ok=True)

# Convert each list of (possibly numpy) numbers into plain Python floats
tracker_serializable = {
    model: [float(x) for x in lst]
    for model, lst in tracker.items()
}

with open(os.path.join(output_dir, "ihdp.json"), "w") as f:
    json.dump(tracker_serializable, f, indent=4)


In [246]:
np.array(tracker['fr']).mean()

3.9843862

In [231]:
np.array(tracker['dr']).mean()

3.998443755227415

In [232]:
np.array(tracker['dragonnet']).mean()

4.0953474

In [242]:
    # Initialize Frengression model
    model = Frengression(x_tr.shape[1], y_tr.shape[1], z_tr.shape[1], 
                         noise_dim=1, num_layer=3, hidden_dim=500, 
                         device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                         x_binary=binary_intervention, z_binary_dims=0)
    

    model.train_y(x_tr, z_tr, y_tr, num_iters=400, lr=1e-4, print_every_iter=100)


    # Sample model distributions
    P0 = model.predict_causal(torch.tensor([0], dtype=torch.int32), sample_size=1000).numpy()
    P1 = model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=1000).numpy().reshape(-1, 1)
    ate_fr = np.mean(P1) - np.mean(P0)

    ate_fr

Epoch 1: loss 6.0942,	loss_y 5.3136, 5.3359, 0.0445,	loss_eta 0.7806, 0.8070, 0.0528
Epoch 100: loss 1.7600,	loss_y 1.2007, 2.0666, 1.7318,	loss_eta 0.5593, 1.0574, 0.9961
Epoch 200: loss 1.2398,	loss_y 0.7024, 1.5397, 1.6746,	loss_eta 0.5374, 1.0438, 1.0129
Epoch 300: loss 1.1299,	loss_y 0.5241, 1.1548, 1.2613,	loss_eta 0.6058, 1.0742, 0.9369
Epoch 400: loss 1.0313,	loss_y 0.4802, 0.9915, 1.0226,	loss_eta 0.5511, 0.9981, 0.8939


4.643131